(cont:sync:sharing)=
# Cooperating Processes and Inter-process Communication

The process abstraction is designed to give the illusion that a program executes on its own
private, isolated (virtual) machine. The OS scheduling mechanism hides the fact that the
physical CPU is, in fact, shared by multiple processes. From the perspective of any given 
process, it appears as though its instructions execute one after the other, in the order 
that they appear in the program, on a dedicated CPU, without interruption or interference 
by the execution of other processes. The process also has sole access to a virtual address
space, where it can store, and operate on, its data -- there is no concern about access to
the process's memory from other processes, or that the process may inadvertently access 
memory that does not belong to it.

But, sometimes we want multiple processes to work together to solve a problem. How can they
cooperate if they are completely isolated from each other? The short answer is, they can't. For processes to cooperate, they need some way to communicate with each other, so that they can exchange information and coordinate their activities. Let's think about some ways that multiple processes can work together, and how they can communicate.

[Unix command pipelines](cont:gs:abstractions:pipes)
  : Each program is designed to do one thing well, and more complex operations can be created by composing multiple simple programs. Here, communication is supported by the `pipe()` system call, which creates a communication channel between processes. With some manipulation of file descriptors, the shell can arrange for the standard output of one process to become the standard input of the next process in the pipeline. The pipe provides both a way to pass data from one process to the next and a way to control the order in which processes execute, because a downstream process cannot read its input until after the previous process in the pipeline has started to produce some output.
  
Complex services
  : A complex service, such as a banking service, may have multiple logical operations that it needs to carry out, such as handling user input, executing account operations like deposits, calculating daily interest, running fraud detection analytics, etc. It is often convenient to implement the service as a collection of processes, with each process carrying out one of the major tasks of the service and the operating system [scheduler](cont:scheduling:scheduling) ensuring that all tasks make progress concurrently. The processes may communicate by reading and writing shared files in the file system. By  
  
Parallel computation
 : In this case, the goal is to complete a single task in less time by using multiple physical compute cores. As a trivial example, consider sorting a very large set of values. On a multi-core machine, you could create one process per core, arrange for each process to sort a portion of the input, and then merge the results together after all the individual sorts were complete. Here, processes might communicate using a combination of files in the file system (e.g., to read the original, unsorted input, and write the intermediate results from each sorting process) and pipes (e.g., to notify the merging process when each sorting process was finished). 
 
 
 
## Shared Memory
To support more efficient, fine-grained inter-process communication, the operating system also provides system calls to let processes explicitly share a portion of their virtual address space. This allows processes to exchange information simply by reading or writing variables that are allocated in the shared part of the address space. There are actually two ways to do this on Unix systems. The first comes from UNIX System V:

- `segment_id = shmget(key, size, shmflg)` : return the identifier of the System V shared memory segment associated with the value of the argument `key`; may be used either to obtain the identifier of a previously created shared memory segment (when `shmflg` is zero and `key` does not have the value `IPC_PRIVATE`), or to create a new shared memory segment.
- `shared_startvaddr = shmat(shmid, *shmaddr, shmflg)` : attach the System V shared memory segment identified by `shmid` to the address space of the calling process at `shmaddr` or an address chosen by the OS if `shmaddr` is `NULL`; return the actual starting virtual address of the shared segment in the process address space. 
- `err = shmdt(shmaddr)` : detach the shared memory segment located at the address specified by `shmaddr` from the address space of the calling process; the to-be-detached segment must be currently attached with `shmaddr` equal to the value returned by the attaching `shmat()` call.

The use of segment identifiers makes it possible for unrelated processes to share memory, however after one process creates a shared memory segment with `shmget`, it needs to communicate the segment id to the other processes that are allowed to share it. One way to do this is by writing the segment id into a file that can be read by the other processes. The other method for requesting shared memory comes from BSD UNIX and requires a parent process to create a shared mapping in its address space before forking child processes. The mapped segments are then shared with child processes, rather than creating a copy in the child address space.

- `shared_startvaddr = mmap(*addr, length, prot, MAP_SHARED | MAP_ANONYMOUS, ...)` : create a new mapping of `length` bytes in the virtual address space of the calling process starting at  `addr` or an address chosen by the OS if `addr` is `NULL`; return the actual starting virtual address of the shared segment in the process address space.

Note that `mmap` is often used to access persistent files using a virtual memory interface rather than the file system `read()` and `write()` interface. In the example above, `MAP_ANONYMOUS` indicates that the mapping is for a non-persistent virtual memory region, which is not backed by any file. (Remember that you can use `man` for the full details of these system calls.)

The shared memory segments are implemented in the operating system using the virtual memory mechanisms. Each process that shares the memory segment has its page tables pointing to the same underlying physical pages, so that any load or store access to a virtual address in the shared segment will be translated to the same physical location in the machine memory.

## Multi-threaded Processes

Recall from {numref}`cont:scheduling:process:threads` that a process can have multiple *threads*, and that all threads in a process share the same virtual address space. This means that multi-threaded processes do not need to do any extra work to communicate using shared memory. The operating system itself is multi-threaded: every user-level process has at least one thread in the kernel, and other threads carry out asynchronous operating system activities, such as writing dirty file system blocks back to the disk. All of these kernel threads share the same operating system code and data structures. We will see later how user-level communication through pipes is implemented in the operating system by threads writing to, and reading from, a shared buffer of memory. In our discussion of concurrency and synchronization, we will use examples in multi-threaded user-level processes, since they can be presented as simple, stand-alone programs. Keep in mind that everything we say about multi-threaded processes also applies to the implementation of the operating system. 

## Coordinating Thread or Process Activities

Regardless of whether we are using multiple processes, or a single multi-threaded process, whether we are implementing user-level programs or the operating system, we need a way to coordinate multiple concurrent activities. 

```{sidebar}
*Concurrency* in computer systems means that the execution of multiple (i.e., two or more) tasks *overlaps in time*. The context switch mechanism gives us concurrent execution of multiple processes, even on a single physical CPU. *Parallelism* means that multiple tasks are executing *at the same time*. Parallelism requires multiple physical CPUs. 
```

*Synchronization* is the mechanism that , since each thread executes independently and asynchronously from the others, and our goal is to control the relative timing of events in one 

